In [ ]:
import pandas as pd
import sklearn.model_selection as sk
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing
from sklearn import metrics
import numpy as np
from sklearn.metrics import confusion_matrix

# Wczytanie i przekształcenie danych
Usunięte zostały wiersze, w których nie było informacji o wydawnictwie.

In [ ]:
data = pd.read_csv("superheroes.csv",)
data = data[data['creator'].notna()]
data = data.fillna(' ')

In [ ]:
superheroes = data[["creator", "history_text", "powers_text", "superpowers", "real_name", "power_score", "teams", "type_race"]]

#utworzenie zmiennej 'joined' z kolumnami zawierającymi wymagane kolumny do klasyfikacji ("power_text", "history_text")
superheroes['joined'] = superheroes["history_text"] + superheroes["powers_text"]

#utworzenie zmiennej 'joined2' rozszerzonej o inne kolumny danych, które mogłby okazać sie pomocne przy klasyfikacji
superheroes['joined2'] = superheroes["history_text"] + superheroes["powers_text"] + superheroes["superpowers"] + superheroes["real_name"] + superheroes["teams"]+ superheroes["type_race"]

#zmiana nazw wszystkich wydawnincw nienależących do Marvel Comics i DC Comics na "Other"
superheroes["creator2"] = [x if x == "Marvel Comics" or x == "DC Comics" else "Other" for x in superheroes['creator'] ]



## Podział próbki na zbiór uczący oraz testowy

In [ ]:
train, test = sk.train_test_split(superheroes, test_size=0.3, random_state=42, shuffle=True)

## Sprawdzenie reprezentantywności próbki
Można zauważyć, że proporcje w każdym zbiorze są do siebie zbliżone więc podział nadaje się do przeprowadzenia klasyfikacji.

In [ ]:
superheroes["creator2"].value_counts()

Marvel Comics    615
DC Comics        444
Other            252
Name: creator2, dtype: int64

In [ ]:
train["creator2"].value_counts()

Marvel Comics    437
DC Comics        305
Other            175
Name: creator2, dtype: int64

In [ ]:
test["creator2"].value_counts()

Marvel Comics    178
DC Comics        139
Other             77
Name: creator2, dtype: int64

## Wektoryzacja

In [ ]:
vectorizer_count = CountVectorizer(stop_words='english')

X_train = vectorizer_count.fit_transform(train.joined)
X_test = vectorizer_count.transform(test.joined)
y_train = train.creator2
y_test = test.creator2

In [ ]:
X_train.shape

(917, 27148)

In [ ]:
vectorizer_count = CountVectorizer(stop_words='english')

X_train2 = vectorizer_count.fit_transform(train.joined2)
X_test2 = vectorizer_count.transform(test.joined2)
y_train2 = train.creator2
y_test2 = test.creator2

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
optimal_alpha = 0.5
NB_optimal = BernoulliNB(alpha=optimal_alpha)

In [ ]:
# fitting the model
NB_optimal.fit(X_train, y_train)

BernoulliNB(alpha=0.5)

In [ ]:
# predict the response
pred = NB_optimal.predict(X_test)

In [ ]:
NB_optimal.score(X_train, y_train)

0.8484187568157033

In [ ]:
y_score = NB_optimal.score(X_test, y_test)
y_score

0.7461928934010152

In [ ]:
print(metrics.classification_report(y_test, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.65      0.93      0.76       139
Marvel Comics       0.88      0.67      0.76       178
        Other       0.79      0.58      0.67        77

     accuracy                           0.75       394
    macro avg       0.77      0.73      0.73       394
 weighted avg       0.78      0.75      0.74       394



In [ ]:
matrix = confusion_matrix(y_test, pred)
pd.DataFrame(matrix)

,0,1,2
0,129,8,2
1,48,120,10
2,23,9,45


Najbardziej optymalnym wspólczynnikiem alpha dla Klasyfikatora Naiwnego Bayesa okzało się alpha = 0.5. Klasyfikator ten uzyskał trafność na poziomie 0.85 na próbie uczącej, oraz 0.75 na próbie testowej. Prezycja dla "DC Comics" czyli ilość dobrze zakfalifikowanych jednostek do odpowiedniej klasy w stosunku do wszystkich zakfalifikowanych do tej klasy jednostek wynosi 0.65 Jest to najniższy wynik, co oznacza, że do tej klasy często kfalifikowane były dane z innych klas. Klasyfikator najczęściej mylił się w przypadku klasy "Other" i przypisywał ją do klasy "DC Comics", dlatego też w tej klasie czułość jest najniższa.  

### Klasyfikacja z uwzględnieniem większej ilości kolumn danych.



In [ ]:
optimal_alpha = 0.4
NB2 = BernoulliNB(alpha=optimal_alpha)

In [ ]:
NB2.fit(X_train2, y_train2)

BernoulliNB(alpha=0.4)

In [ ]:
pred = NB2.predict(X_test2)

In [ ]:
NB2.score(X_train2, y_train2)

0.8495092693565977

In [ ]:
y_score = NB2.score(X_test2, y_test2)
y_score

0.7690355329949239

In [ ]:
print(metrics.classification_report(y_test2, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.72      0.88      0.80       139
Marvel Comics       0.87      0.70      0.78       178
        Other       0.68      0.73      0.70        77

     accuracy                           0.77       394
    macro avg       0.76      0.77      0.76       394
 weighted avg       0.78      0.77      0.77       394



In [ ]:
matrix = confusion_matrix(y_test2, pred)
pd.DataFrame(matrix)

,0,1,2
0,123,9,7
1,35,124,19
2,12,9,56


Dla większej ilości przyjętych kolumn danych trafność dla próby testowej nieznacznie wzrosła, jednakże wszystkie wyniki są bardzo zlbiżone.

### 30 słów  mających największy wpływ przy klasyfikacji postaci
#### Dla kolumn "powers_text" oraz "history_text"

In [ ]:
def most_informative_feature_for_class(vectorizer, classifier, classlabel, n=30):
    labelid = list(classifier.classes_).index(classlabel)
    feature_names = vectorizer.get_feature_names()
    topn = sorted(zip(classifier.coef_[labelid], feature_names))[-n:]
    features = []
    for coef, feat in topn:
        #print(classlabel, feat, coef)
        features.append(feat)
    #print(features)
    return features


In [ ]:
DC_Comics = most_informative_feature_for_class(vectorizer_count, NB_optimal,"DC Comics", n=30)
DC = pd.DataFrame(DC_Comics, columns=["DC Comics"])

Marvel_Comics = most_informative_feature_for_class(vectorizer_count, NB_optimal,"Marvel Comics", n=30)
Marvel = pd.DataFrame(Marvel_Comics, columns=["Marvel Comics"])

Oth = most_informative_feature_for_class(vectorizer_count, NB_optimal,"Other", n=30)
Othe = pd.DataFrame(Oth, columns=["Other"])

In [ ]:
all = pd.concat([DC, Marvel, Othe], axis = 1)
all

,DC Comics,Marvel Comics,Other
0,timebroker,abilities,porr
1,harsh,liar,volantis
2,kept,restorations,boasts
3,petrovna,tampering,heresy
4,deactivate,dyed,ferrous
5,heightens,heightens,foreign
6,stirs,lawyers,heightens
7,batrope,timebroker,lawyers
8,wraithlike,wink,awakens
9,liar,deactivate,faora


#### Przy uwzględnieniu większej ilości kolumn

In [ ]:
DC_Comics = most_informative_feature_for_class(vectorizer_count, NB2,"DC Comics", n=30)
DC = pd.DataFrame(DC_Comics, columns=["DC Comics"])

Marvel_Comics = most_informative_feature_for_class(vectorizer_count, NB2,"Marvel Comics", n=30)
Marvel = pd.DataFrame(Marvel_Comics, columns=["Marvel Comics"])

Oth = most_informative_feature_for_class(vectorizer_count, NB2,"Other", n=30)
Othe = pd.DataFrame(Oth, columns=["Other"])

In [ ]:
all = pd.concat([DC, Marvel, Othe], axis = 1)
all

,DC Comics,Marvel Comics,Other
0,used,world,force
1,use,using,life
2,force,death,like
3,eventually,battle,new
4,city,killed,mind
5,known,master,resistance
6,earth,weapons,healing
7,stealth,used,weapon
8,based,life,able
9,weapon,eventually,accelerated


# K Neighbours Classifier

Nearest Neighbours = 7

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.score(X_train, y_train)

0.7535441657579062

In [ ]:
knn.score(X_test, y_test)

0.6776649746192893

In [ ]:
pred = knn.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.59      0.86      0.70       139
Marvel Comics       0.76      0.78      0.77       178
        Other       1.00      0.12      0.21        77

     accuracy                           0.68       394
    macro avg       0.78      0.58      0.56       394
 weighted avg       0.75      0.68      0.63       394



In [ ]:
matrix = confusion_matrix(y_test, pred)
pd.DataFrame(matrix)

,0,1,2
0,119,20,0
1,39,139,0
2,43,25,9


Najlepszym parameterm w tym klasyfikatorze okazało się 7 sąsiadów. Trafność dla zbioru uczącego wynosi 0.75, natomiast dla testowego jest trochę gorsza i wynosi 0.68. Najgorszą trafnosć ma zbiór "Other", czyli dane nienależące do "DC Comics" oraz "Marvel Comics". Może to wynikać z tego, że ilość tych danych jest najmniejsza w porównaniu do dwóch pozostałych wydawnictw, a także z tego, że trafiają tam pomieszane dane z różnych wydawnictw. Widać również, że dane z tego zbioru najczęściej przypisywane były do "DC Comics", w którym ponownie jest najwięcej fałszywych pozytywów.

### Klasyfikacja z uwzględnieniem większej ilości danych

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors=4)
knn2.fit(X_train2, y_train2)

KNeighborsClassifier(n_neighbors=4)

In [ ]:
knn2.score(X_train2, y_train2)

0.7241003271537623

In [ ]:
knn2.score(X_test2, y_test2)

0.6802030456852792

In [ ]:
pred = knn2.predict(X_test2)

In [ ]:
print(metrics.classification_report(y_test2, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.67      0.72      0.69       139
Marvel Comics       0.67      0.85      0.75       178
        Other       0.89      0.22      0.35        77

     accuracy                           0.68       394
    macro avg       0.74      0.60      0.60       394
 weighted avg       0.71      0.68      0.65       394



In [ ]:
matrix = confusion_matrix(y_test2, pred)
pd.DataFrame(matrix)

,0,1,2
0,100,38,1
1,26,151,1
2,23,37,17


Przy uwzględineniu większej ilości danych najlepszym parametrem okazało się 4 sąsiadów. Trafność klasyfikatora pogorszyła się dla zbioru uczacego, a polepszyła dla zbioru testowego, jednakże bardzo nieznacznie. Dla zbioru uczącego wyniosła 0.72, a testowego 0.68.

# Nearest Centroid - Rocchio

In [ ]:
from sklearn.neighbors import NearestCentroid

In [ ]:
clf = NearestCentroid(metric='euclidean')

In [ ]:
clf.fit(X_train, y_train)

NearestCentroid()

In [ ]:
clf.score(X_train, y_train)

0.6030534351145038

In [ ]:
clf.score(X_test, y_test)

0.5736040609137056

In [ ]:
pred = clf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.46      0.92      0.62       139
Marvel Comics       0.84      0.54      0.66       178
        Other       0.67      0.03      0.05        77

     accuracy                           0.57       394
    macro avg       0.66      0.50      0.44       394
 weighted avg       0.67      0.57      0.52       394



In [ ]:
matrix = confusion_matrix(y_test, pred)
pd.DataFrame(matrix)

,0,1,2
0,128,11,0
1,81,96,1
2,68,7,2


Najlepsze wyniki klasyfikacji dawało wykorzystanie standardowej odległości euklidesowej. Klasyfikator Nearest Centroid okazał się najgorszym wykorzystanym do tej pory klasyfikatorem. Wyniki, które zostały uzyskane (trafność dla zbioru uczącego = 0.60, a dla testowego = 0.57) raczej są niezadowalające. Klasyfikator ten zupełnie nie poradził sobie z klasyfikacją dla zbioru "Other". Do zbioru "DC Comics" ponownie trafiło najwięcej fałszywych pozytywów.

### Dla większej ilości danych

In [ ]:
clf2 = NearestCentroid()

In [ ]:
clf2.fit(X_train2, y_train2)

NearestCentroid()

In [ ]:
clf2.score(X_train2, y_train2)

0.613958560523446

In [ ]:
clf2.score(X_test2, y_test2)

0.5939086294416244

In [ ]:
pred = clf2.predict(X_test2)

In [ ]:
print(metrics.classification_report(y_test2, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.48      0.95      0.64       139
Marvel Comics       0.87      0.56      0.68       178
        Other       0.60      0.04      0.07        77

     accuracy                           0.59       394
    macro avg       0.65      0.51      0.46       394
 weighted avg       0.68      0.59      0.55       394



In [ ]:
matrix = confusion_matrix(y_test2, pred)
pd.DataFrame(matrix)

,0,1,2
0,132,7,0
1,77,99,2
2,66,8,3


Przy wykorzystaniu większej ilości danych wyniki nieznacznie się polepszyły, jednak dalej wypadają znacząco gorzej na tle poprzednich klasyfikatorów.

# LinearSVC

In [ ]:
 from sklearn.svm import LinearSVC

In [ ]:
lin = LinearSVC(penalty='l1', loss='squared_hinge', dual=False, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, max_iter=1000, random_state=None)

In [ ]:
lin.fit(X_train, y_train)

In [ ]:
lin.score(X_train, y_train)

1.0

In [ ]:
lin.score(X_test, y_test)

0.7741116751269036

In [ ]:
pred = lin.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.79      0.71      0.75       139
Marvel Comics       0.78      0.89      0.83       178
        Other       0.73      0.62      0.67        77

     accuracy                           0.77       394
    macro avg       0.77      0.74      0.75       394
 weighted avg       0.77      0.77      0.77       394



In [ ]:
matrix = confusion_matrix(y_test, pred)
pd.DataFrame(matrix)

,0,1,2
0,99,29,11
1,13,158,7
2,14,15,48


Dla przyjętych parametrow liniowego klasyfikatora SVM trafność dla próby uczącej wyniosła 1, co jest idelanym wynikiem. Jednakże, dla próby testowej trafność jest gorsza i wynosi 0.77. Prezycja dla każdej z grup jest na zbliżonym poziomie. Żadna z grup nie ma szczególnie dużej ilości fałszywych pozytyów.

### Klasyfikacja z uwzględnieniem większej ilości danych

In [ ]:
lin2 = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=0.0001, C=8.0, multi_class='crammer_singer', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, max_iter=10, random_state=None)

In [ ]:
lin2.fit(X_train2, y_train2)

In [ ]:
lin2.score(X_train2, y_train2)

1.0

In [ ]:
lin2.score(X_test2, y_test2)

0.8147208121827412

In [ ]:
pred = lin2.predict(X_test2)

In [ ]:
print(metrics.classification_report(y_test2, pred, target_names=["DC Comics" ,"Marvel Comics", "Other"]))

               precision    recall  f1-score   support

    DC Comics       0.81      0.81      0.81       139
Marvel Comics       0.85      0.89      0.87       178
        Other       0.71      0.65      0.68        77

     accuracy                           0.81       394
    macro avg       0.79      0.78      0.79       394
 weighted avg       0.81      0.81      0.81       394



In [ ]:
matrix = confusion_matrix(y_test, pred)
pd.DataFrame(matrix)

,0,1,2
0,112,15,12
1,11,159,8
2,15,12,50


Przy uwzględnieniu większej ilości danych trafność klasyfikaci okazała się lepsza, ponieważ na zbiorze uczącym wyninosła 0.81. Czułość dla klasy "Marvel Comics" wynosi aż 0.89 co oznacza, że prawie 90% bohaterów z tej klasy została dobrze zaklasyfikowana.

## 30 najważniejszych słów dla klasyfikacji

In [ ]:
DC_Comics = most_informative_feature_for_class(vectorizer_count, lin,"DC Comics", n=30)
DC = pd.DataFrame(DC_Comics, columns=["DC Comics"])

Marvel_Comics = most_informative_feature_for_class(vectorizer_count, lin,"Marvel Comics", n=30)
Marvel = pd.DataFrame(Marvel_Comics, columns=["Marvel Comics"])

Oth = most_informative_feature_for_class(vectorizer_count, lin,"Other", n=30)
Othe = pd.DataFrame(Oth, columns=["Other"])

In [ ]:
all = pd.concat([DC, Marvel, Othe], axis = 1)
all

,DC Comics,Marvel Comics,Other
0,suffering,niobium,cyclotron
1,joyride,tigers,ropes
2,egghead,algeria,releasing
3,illusion,verbal,maturation
4,krypt,manipulations,fearsome
5,targeting,indulging,viii
6,funnier,redd,marred
7,bovo,callous,alley
8,armory,limits,updated
9,silhouettes,martina,sending


In [ ]:
DC_Comics = most_informative_feature_for_class(vectorizer_count, lin2,"DC Comics", n=30)
DC = pd.DataFrame(DC_Comics, columns=["DC Comics"])

Marvel_Comics = most_informative_feature_for_class(vectorizer_count, lin2,"Marvel Comics", n=30)
Marvel = pd.DataFrame(Marvel_Comics, columns=["Marvel Comics"])

Oth = most_informative_feature_for_class(vectorizer_count, lin2,"Other", n=30)
Othe = pd.DataFrame(Oth, columns=["Other"])

In [ ]:
all = pd.concat([DC, Marvel, Othe], axis = 1)
all

,DC Comics,Marvel Comics,Other
0,boy,laufey,castiel
1,underwater,longevity,antagonists
2,eternal,lorelei,evolved
3,roy,america,creation
4,grayven,weapons,series
5,gotham,spider,believed
6,hiro,alien,mall
7,cash,stark,guy
8,lantern,colt,hair
9,league,order,dean


#  RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier #98 74

In [ ]:
RF = RandomForestClassifier(n_estimators = 200, max_depth=70, min_samples_leaf=2)

In [ ]:
RF.fit(X_train, y_train)

RandomForestClassifier(max_depth=70, min_samples_leaf=2, n_estimators=200)

In [ ]:
pred=RF.predict(X_test)

In [ ]:
#y_pred

In [ ]:
RF.score(X_train, y_train)

0.9858233369683751

In [ ]:
RF.score(X_test, y_test)

0.7538071065989848

In [ ]:
print(metrics.classification_report(y_test, pred, target_names=["Marvel Comics", "DC Comics" , "Other"]))

               precision    recall  f1-score   support

Marvel Comics       0.88      0.74      0.80       139
    DC Comics       0.68      0.98      0.80       178
        Other       0.94      0.22      0.36        77

     accuracy                           0.75       394
    macro avg       0.83      0.65      0.65       394
 weighted avg       0.80      0.75      0.72       394



In [ ]:
matrix = confusion_matrix(y_test, pred)
pd.DataFrame(matrix)

,0,1,2
0,103,36,0
1,2,175,1
2,12,48,17


Klasyfikator uzyskał bardzo dobry wynik trafności na zbiorze uczącym = 0.99, natomiast na zbiorze testowym dość znacząco się pogorszył (trafność = 0.75). Można również zauważyć, że błędnie do zbioru Other zaklasyfikował tylko jedego superbohatera, więc precyzja dla tej grupy jeste bardzo wysoka (0.94). Jednak mimo wysokiej precyzji zbiór ten ma też najgorszą (i widocznie odstającą od dwóch pozostałych grup) czułość wynoszącą 0.22 - większość superbohaterów należących do tego zbioru zostało błędnie przypisanych do innych wydawnictw. Natomiast czułość dla wydawnictwa DC Comics jest bardzo wysoka (0.98) i prawie wszystkie jednostki (z wyjątkiem 3) tej grupy zostały prawidłowo zaklasyfikowane.

### Klasyfikacja z uwzględnieniem większej ilości danych

In [ ]:
RF2 = RandomForestClassifier(n_estimators = 100, max_depth=70, min_samples_leaf=2)

In [ ]:
#n_estimators = 800, max_depth=70, min_samples_leaf=2)
#n_estimators = 1000, max_depth=70, min_samples_leaf=2) - 79

In [ ]:
RF2.fit(X_train2, y_train2)

RandomForestClassifier(max_depth=70, min_samples_leaf=2)

In [ ]:
pred=RF2.predict(X_test2)

In [ ]:
RF2.score(X_train2, y_train2)

0.9792802617230099

In [ ]:
RF2.score(X_test2, y_test2)

0.7893401015228426

In [ ]:
print(metrics.classification_report(y_test2, pred, target_names=["Marvel Comics", "DC Comics" , "Other"]))

               precision    recall  f1-score   support

Marvel Comics       0.90      0.80      0.85       139
    DC Comics       0.71      0.98      0.83       178
        Other       0.96      0.32      0.49        77

     accuracy                           0.79       394
    macro avg       0.86      0.70      0.72       394
 weighted avg       0.83      0.79      0.77       394



In [ ]:
matrix = confusion_matrix(y_test2, pred)
pd.DataFrame(matrix)

,0,1,2
0,111,28,0
1,2,175,1
2,10,42,25


Przy większej ilości wykorzystanych danych trafność na zbiorze testowym polepszyła się i wynosi 0.79.

# Porównanie i podsumowanie

### Dla kolumn "histotry_text", "powers_text"

In [ ]:
wartosci = {
  "Naive Bayes": [0.85, 0.75],
  "K Neighbours Classifier": [0.75, 0.68],
  "Nearest Centroid": [0.60, 0.57],
  "Linear SVC": [1, 0.77],
  "Random Forrest Classifier": [0.99, 0.75],
}

In [ ]:
pd.DataFrame.from_dict(wartosci, orient='index', columns= ["Uczący", "Testowy"])

,Uczący,Testowy
Naive Bayes,0.85,0.75
K Neighbours Classifier,0.75,0.68
Nearest Centroid,0.60,0.57
Linear SVC,1.00,0.77
Random Forrest Classifier,0.99,0.75


Najlepszym klasyfikatorem okazał się LinearSVC, natomiast najgorszym NearestCentroid.  

### Klasyfikacja z uwzględnieniem większej ilości danych

In [ ]:
wartosci2 = {
  "Naive Bayes": [0.85, 0.77],
  "K Neighbours Classifier": [0.73, 0.68],
  "Nearest Centroid": [0.61, 0.59],
  "Linear SVC": [1, 0.81],
  "Random Forrest Classifier": [0.98, 0.79],
}

In [ ]:
pd.DataFrame.from_dict(wartosci2, orient='index', columns= ["Uczący", "Testowy"])

,Uczący,Testowy
Naive Bayes,0.85,0.77
K Neighbours Classifier,0.73,0.68
Nearest Centroid,0.61,0.59
Linear SVC,1.00,0.81
Random Forrest Classifier,0.98,0.79


Przy uwzlędnieniu większej ilości danych, najlepszym klasyfikatorem również okazał się Linear SVC. Wyniki w prawie wszystkich klasyfikatorach polepszyły się, aczkolwiek są bardzo zbliżone do poprzednich. Nearest Centroid niezmiennie poradził sobie najgorzej z klasyfikacją postaci. Należy jednak zaznaczyć, że nie zostały wybrane wszystkie pozostałe kolumny, tylko te, które w subiektywny sposób zostały uznane za istotne.